In [1]:
import requests
import datetime
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# 한국은행 기준금리

In [4]:
base_url= "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
res = requests.get(base_url, 'html.parser')
soup= BeautifulSoup(res.text, 'lxml')

In [5]:
dict_list= []

for i in range(45):
    dict_list.append({
        'date': pd.to_datetime(datetime.datetime.strptime(soup.find_all('td')[3*i].text\
             + soup.find_all('td')[3*i+1].text, '%Y%m월 %d일')),
        'rate': float(soup.find_all('td')[3*i+2].text)
    })

In [6]:
pd_result= pd.DataFrame(dict_list)

# 원하는 전체 date range
idx= pd.date_range(pd_result.date.min(), pd_result.date.max())
idx= pd.Series(idx)

In [7]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
baserate= pd.concat([pd.DataFrame({'date': idx[~idx.isin(pd_result.date)], 'rate': np.nan}),pd_result])\
    .sort_values('rate').reset_index(drop=True).ffill(axis=0)

In [10]:
baserate

,date,rate
0,2020-05-28,0.50
1,2020-03-17,0.75
2,2021-08-26,0.75
3,2021-11-25,1.00
4,2016-06-09,1.25
...,...,...
7073,2022-11-19,5.25
7074,2022-11-20,5.25
7075,2022-11-21,5.25
7076,2022-11-22,5.25


In [26]:
baserate.to_csv('./baserate.csv')

# 콜금리

In [15]:
base_url2= "https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}"

In [16]:
base_url2= "https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}"
page_num= 384


In [17]:
soup_list2= []

for i in range(1,page_num+1):
    soup_list2.append(BeautifulSoup(requests.get(base_url2.format(i), 'html.parser').text,'lxml').find_all('td'))

In [18]:
result_list2= []

for lists in soup_list2:
    for i in range(len(lists)//4):
        result_list2.append({
            'date': lists[4*i].text.strip(),
            'rate': float(lists[4*i+1].text)
        })

In [19]:
pd_result2= pd.DataFrame(result_list2)
pd_result2.date= pd_result2.date.astype('datetime64')

In [20]:
# 원하는 전체 date range
idx2= pd.date_range(pd_result2.date.min(), pd_result2.date.max())
idx2= pd.Series(idx2)

In [21]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
callrate= pd.concat([pd.DataFrame({'date': idx2[~idx2.isin(pd_result2.date)], 'rate': np.nan}),pd_result2])\
    .sort_values('date').reset_index(drop=True).ffill(axis=0)[1:]

In [22]:
callrate

,date,rate
1,2011-12-31,3.29
2,2012-01-01,3.29
3,2012-01-02,3.25
4,2012-01-03,3.25
5,2012-01-04,3.24
...,...,...
3972,2022-11-14,3.07
3973,2022-11-15,3.03
3974,2022-11-16,3.03
3975,2022-11-17,3.02


In [23]:
callrate.to_csv('../data/call_rate.csv')